In [2]:
import pandas as pd

In [3]:
# 1. Load the datasets
games = pd.read_csv('games.csv')
teams = pd.read_csv('teams.csv')
details = pd.read_csv('games_details.csv', low_memory=False) # low_memory=False helps with large files

In [4]:
# 2. Convert Date columns to actual datetime objects (Crucial for trends!)
# Depending on your specific dataset version, the column might be 'GAME_DATE_EST' or 'GAME_DATE'
if 'GAME_DATE_EST' in games.columns:
    games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE_EST'])
else:
    games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])

In [5]:
# Sort by date helps with plotting later
games = games.sort_values('GAME_DATE')

print("Data Loaded Successfully!")
print(f"Games Shape: {games.shape}")
print(f"Teams Shape: {teams.shape}")

Data Loaded Successfully!
Games Shape: (26651, 22)
Teams Shape: (30, 14)


In [6]:
# 1. Create a smaller "lookup" table from teams 
# We only want ID, Abbreviation (LAL), Nickname (Lakers), and City (Los Angeles)
teams_subset = teams[['TEAM_ID', 'ABBREVIATION', 'NICKNAME', 'CITY']]

In [7]:
# 2. Merge for HOME Team
# We match 'HOME_TEAM_ID' from games with 'TEAM_ID' from teams
games_merged = pd.merge(games, teams_subset, 
                        left_on='HOME_TEAM_ID', 
                        right_on='TEAM_ID', 
                        how='left')

# Rename the columns so we know they belong to the HOME team
games_merged.rename(columns={
    'ABBREVIATION': 'HOME_TEAM_ABBREV',
    'NICKNAME': 'HOME_TEAM_NAME',
    'CITY': 'HOME_TEAM_CITY'
}, inplace=True)

# Drop the extra 'TEAM_ID' column that came from the merge (we don't need it anymore)
games_merged.drop('TEAM_ID', axis=1, inplace=True)

In [8]:
# 3. Merge for AWAY (Visitor) Team
# Now we match 'VISITOR_TEAM_ID' from games with 'TEAM_ID' from teams
games_merged = pd.merge(games_merged, teams_subset, 
                        left_on='VISITOR_TEAM_ID', 
                        right_on='TEAM_ID', 
                        how='left')

In [9]:
# Rename the columns so we know they belong to the AWAY team
games_merged.rename(columns={
    'ABBREVIATION': 'AWAY_TEAM_ABBREV',
    'NICKNAME': 'AWAY_TEAM_NAME',
    'CITY': 'AWAY_TEAM_CITY'
}, inplace=True)

In [10]:
# Drop the extra 'TEAM_ID' column again
games_merged.drop('TEAM_ID', axis=1, inplace=True)

In [11]:
# Check if it worked
print("Merge Successful!")
print(games_merged[['GAME_ID', 'HOME_TEAM_NAME', 'AWAY_TEAM_NAME', 'PTS_home', 'PTS_away']].head())

Merge Successful!
    GAME_ID HOME_TEAM_NAME AWAY_TEAM_NAME  PTS_home  PTS_away
0  10300001           Jazz      Mavericks      90.0      85.0
1  10300002      Grizzlies          Bucks     105.0      94.0
2  10300010        Wizards         Knicks     104.0      86.0
3  10300009          Kings       Clippers     101.0      82.0
4  10300005  Trail Blazers        Rockets     104.0      80.0


In [12]:
# 1. Grab just the ID and Date from our new games file
games_dates = games_merged[['GAME_ID', 'GAME_DATE', 'SEASON']]

In [13]:
# 2. Merge them into the player details file
# This connects every stat line (points, rebounds) to a specific date
player_stats_master = pd.merge(details, games_dates, on='GAME_ID', how='left')

In [14]:
# 3. Clean up: Remove rows where players didn't play (MIN is NaN)
player_stats_master = player_stats_master.dropna(subset=['MIN'])

In [15]:
print("Player Stats Connected to Dates!")
print(player_stats_master[['PLAYER_NAME', 'GAME_DATE', 'PTS', 'REB']].head())

Player Stats Connected to Dates!
      PLAYER_NAME  GAME_DATE   PTS  REB
0  Romeo Langford 2022-12-22   2.0  2.0
1   Jeremy Sochan 2022-12-22  23.0  9.0
2    Jakob Poeltl 2022-12-22  13.0  4.0
3   Devin Vassell 2022-12-22  10.0  9.0
4       Tre Jones 2022-12-22  19.0  2.0


In [16]:
# Save to your current folder
games_merged.to_csv('master_games.csv', index=False)
player_stats_master.to_csv('master_player_stats.csv', index=False)

print("SUCCESS: Phase 1 Complete. Files 'master_games.csv' and 'master_player_stats.csv' created.")

SUCCESS: Phase 1 Complete. Files 'master_games.csv' and 'master_player_stats.csv' created.


In [2]:
import pandas as pd
import plotly.express as px

# 1. Load your clean Master Games file
df = pd.read_csv('master_games.csv')

# 2. Calculate Total Points per Game (Home Score + Away Score)
# We want to see how much SCORING happens in a single game on average
df['TOTAL_GAME_POINTS'] = df['PTS_home'] + df['PTS_away']

# 3. Group by Season to get the Average Score
season_trends = df.groupby('SEASON')['TOTAL_GAME_POINTS'].mean().reset_index()

# Sort by Season so the line goes from left (old) to right (new)
season_trends = season_trends.sort_values('SEASON')

# 4. Plot the Interactive Chart using Plotly
fig = px.line(season_trends, 
              x='SEASON', 
              y='TOTAL_GAME_POINTS',
              markers=True,
              title='<b>Evolution of NBA Scoring (2004 - Present)</b>',
              labels={'TOTAL_GAME_POINTS': 'Average Points per Game', 'SEASON': 'Season'},
              template='plotly_dark') # Dark mode looks professional

# Show the Average Points on hover
fig.update_traces(hovertemplate='Season: %{x}<br>Avg Points: %{y:.1f}')

fig.show()

In [7]:
import pandas as pd
import plotly.express as px

# 1. Load ranking file to find Conference info
rankings = pd.read_csv('ranking.csv')

# 2. Extract unique mapping of TEAM_ID -> CONFERENCE
# We only need one entry per team to know if they are East or West
conf_map = rankings[['TEAM_ID', 'CONFERENCE']].drop_duplicates()

# Filter to ensure we only get valid East/West labels
conf_map = conf_map[conf_map['CONFERENCE'].isin(['East', 'West'])]

# 3. Merge this Conference info into our main dataframe
# We match 'HOME_TEAM_ID' from games with 'TEAM_ID' from our new map
df_conf = pd.merge(df, conf_map, left_on='HOME_TEAM_ID', right_on='TEAM_ID', how='left')

# 4. Calculate Wins
# Logic: If Home Team points > Away Team points, it's a Home Win
home_wins = df_conf[df_conf['PTS_home'] > df_conf['PTS_away']]

# Group by Season and Conference to count total wins
conf_wins = home_wins.groupby(['SEASON', 'CONFERENCE']).size().reset_index(name='WINS')

# 5. Plot the Chart
fig2 = px.bar(conf_wins, 
              x='SEASON', 
              y='WINS', 
              color='CONFERENCE',
              title='<b>Conference Dominance: Total Home Wins by Season</b>',
              labels={'WINS': 'Total Wins', 'SEASON': 'Season'},
              color_discrete_map={'East': '#0051B4', 'West': '#C9082A'}, 
              barmode='group',
              template='plotly_dark')

fig2.show()

In [5]:
# 1. Load ranking file to find Conference info
rankings = pd.read_csv('ranking.csv')

# 2. Extract unique mapping of TEAM_ID -> CONFERENCE
# We only need one entry per team to know if they are East or West
conf_map = rankings[['TEAM_ID', 'CONFERENCE']].drop_duplicates()

# Filter out strictly valid conferences just in case (East/West)
conf_map = conf_map[conf_map['CONFERENCE'].isin(['East', 'West'])]

# 3. Merge this Conference info into our main dataframe
# We match 'HOME_TEAM_ID' from games with 'TEAM_ID' from our new map
df_conf = pd.merge(df, conf_map, left_on='HOME_TEAM_ID', right_on='TEAM_ID', how='left')

# 4. Calculate Wins
# Logic: If Home Team points > Away Team points, it's a Home Win
home_wins = df_conf[df_conf['PTS_home'] > df_conf['PTS_away']]

# Group by Season and Conference
conf_wins = home_wins.groupby(['SEASON', 'CONFERENCE']).size().reset_index(name='WINS')

# 5. Plot the Chart
fig2 = px.bar(conf_wins, 
              x='SEASON', 
              y='WINS', 
              color='CONFERENCE',
              title='<b>Conference Dominance: Total Home Wins by Season</b>',
              labels={'WINS': 'Total Wins', 'SEASON': 'Season'},
              color_discrete_map={'East': '#0051B4', 'West': '#C9082A'}, 
              barmode='group',
              template='plotly_dark')

fig2.show()

In [6]:
# --- STEP 3: Home Court Advantage Trend ---

# 1. Create a "Home Win" flag (1 if Home won, 0 if Away won)
# We use our master dataframe 'df' (or reload master_games.csv if needed)
df['IS_HOME_WIN'] = (df['PTS_home'] > df['PTS_away']).astype(int)

# 2. Calculate Win % per Season
# The mean of a 0/1 column gives us the percentage!
home_advantage = df.groupby('SEASON')['IS_HOME_WIN'].mean().reset_index()

# Convert to percentage for display (e.g., 0.6 -> 60%)
home_advantage['WIN_PERCENTAGE'] = home_advantage['IS_HOME_WIN'] * 100

# 3. Plot the Trend
fig3 = px.line(home_advantage, 
               x='SEASON', 
               y='WIN_PERCENTAGE',
               markers=True,
               title='<b>Is Home Court Advantage Disappearing? (Home Win %)</b>',
               labels={'WIN_PERCENTAGE': 'Home Team Win %', 'SEASON': 'Season'},
               template='plotly_dark')

# Add a reference line at 50% (where it would be if there was NO advantage)
fig3.add_hline(y=50, line_dash="dash", line_color="green", annotation_text="Neutral (50%)")

fig3.update_traces(line_color='#00CC96') # A nice teal color
fig3.update_yaxes(range=[40, 70]) # Zoom in to see the trend clearly

fig3.show()

In [8]:
# --- STEP 4: Player Performance Analysis ---

# 1. Load the Player Stats Master file
player_df = pd.read_csv('master_player_stats.csv')

# 2. Filter for a specific season (e.g., 2021) to see active stars
# You can change this year to 2010 to see Kobe/LeBron era, or 2021 for Giannis/Jokic
season_stats = player_df[player_df['SEASON'] == 2021]

# 3. Aggregate Stats by Player (Average per game)
# We group by Player Name and take the MEAN of their stats
top_players = season_stats.groupby('PLAYER_NAME')[['PTS', 'REB', 'AST']].mean().reset_index()

# 4. Filter for "Qualified" players only
# (Remove players who only played garbage time with 0 points)
top_players = top_players[top_players['PTS'] > 10] # Only show players averaging > 10 PPG

# 5. Create the Interactive Scatter Plot
fig4 = px.scatter(top_players, 
                  x='PTS', 
                  y='REB', 
                  size='PTS', # Bubble size = Scoring ability
                  color='AST', # Color = Playmaking ability
                  hover_name='PLAYER_NAME',
                  title='<b>Offensive Threats: Scoring vs. Rebounding (2021)</b>',
                  labels={'PTS': 'Points per Game', 'REB': 'Rebounds per Game', 'AST': 'Assists'},
                  template='plotly_dark',
                  color_continuous_scale='Viridis') # Nice blue-green-yellow gradient

fig4.show()

/tmp/ipykernel_44499/1427784679.py:4: DtypeWarning:

Columns (6,7,9) have mixed types. Specify dtype option on import or set low_memory=False.

